In [8]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from feature_engine.encoding import RareLabelEncoder
from feature_engine.encoding import CountFrequencyEncoder
from feature_engine.outliers import Winsorizer
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.transformation import LogCpTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import HuberRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

In [2]:
tb_data = pd.read_csv('../data/train.csv', index_col='id')
tb_data

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,A,B,A,A,B,D,A,E,C,I,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
2,B,A,A,A,B,B,A,E,A,F,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
3,A,A,A,C,B,D,A,B,C,N,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
4,A,A,A,C,B,D,A,E,G,K,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
6,A,B,A,A,B,B,A,E,C,F,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,A,B,A,C,B,B,A,E,E,L,...,0.269578,0.258655,0.363598,0.300619,0.340516,0.235711,0.383477,0.215227,0.793630,8.343538
499996,A,B,A,C,B,B,A,E,E,L,...,0.197211,0.257024,0.574304,0.227035,0.322583,0.286094,0.324874,0.306933,0.230902,7.851861
499997,A,B,A,C,B,B,A,E,C,M,...,0.449482,0.386172,0.476217,0.135947,0.502730,0.235788,0.316671,0.250286,0.349041,7.600558


In [3]:
range_scale = MinMaxScaler()
X = tb_data.loc[:, :'cont13']
Y = tb_data['target']

In [4]:
cat_columns = X.select_dtypes(include=object).columns.to_list()
cont_columns = X.select_dtypes(include=np.number).columns.to_list()
X[cont_columns] = range_scale.fit_transform(X[cont_columns])
X

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
1,A,B,A,A,B,D,A,E,C,I,...,0.114564,0.855691,0.359931,0.643496,0.882829,0.811322,0.685241,0.654854,0.913387,0.785206
2,B,A,A,A,B,B,A,E,A,F,...,0.115723,0.465907,0.288058,0.084307,0.583471,0.526128,0.570654,0.695283,0.343629,0.908989
3,A,A,A,C,B,D,A,B,C,N,...,0.129890,0.884880,0.310331,0.753604,0.852951,0.836672,0.210755,0.648330,0.904611,0.936786
4,A,A,A,C,B,D,A,E,G,K,...,0.721352,0.902538,0.509813,0.239914,0.855406,0.802359,0.448007,0.650962,0.843251,0.638256
6,A,B,A,A,B,B,A,E,C,F,...,0.111686,0.415176,0.630849,0.140716,0.432551,0.432796,0.685273,0.635506,0.360980,0.195151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499993,A,B,A,C,B,B,A,E,E,L,...,0.719690,0.315305,0.204602,0.187108,0.293608,0.297509,0.181488,0.327877,0.228370,0.888255
499996,A,B,A,C,B,B,A,E,E,L,...,0.729014,0.251359,0.203048,0.441633,0.220760,0.277562,0.233423,0.267703,0.323208,0.101727
499997,A,B,A,C,B,B,A,E,C,M,...,0.134425,0.474276,0.326122,0.323148,0.130585,0.477937,0.181568,0.259280,0.264627,0.266850


In [5]:
base_line = Pipeline(steps=[('rare', RareLabelEncoder(n_categories=3,
                                                      variables=cat_columns)),
                            ('frequancy', CountFrequencyEncoder(encoding_method='frequency',
                                                                variables=cat_columns)),
                            ('winsorizer', Winsorizer(capping_method='iqr', tail='both',
                                                      variables=cont_columns)),
                            ('logit', LogCpTransformer(C=0.001, variables=['cont3', 'cont4', 'cont5', 'cont7',
                                                                           'cont11', 'cont12', 'cont13'])),
                            ('yeo_johnson', YeoJohnsonTransformer(variables=['cont0', 'cont1', 'cont2', 'cont6',
                                                                             'cont8', 'cont9', 'cont10'
                                                                             ])),
                            ('feature_selection', SelectKBest(score_func=mutual_info_regression, k=12))
                            ])
base_line.get_params()

{'memory': None,
 'steps': [('rare', RareLabelEncoder(n_categories=3,
                    variables=['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5',
                               'cat6', 'cat7', 'cat8', 'cat9'])),
  ('frequancy',
   CountFrequencyEncoder(encoding_method='frequency',
                         variables=['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5',
                                    'cat6', 'cat7', 'cat8', 'cat9'])),
  ('winsorizer',
   Winsorizer(capping_method='iqr', tail='both',
              variables=['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
                         'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11',
                         'cont12', 'cont13'])),
  ('logit',
   LogCpTransformer(C=0.001,
                    variables=['cont3', 'cont4', 'cont5', 'cont7', 'cont11',
                               'cont12', 'cont13'])),
  ('yeo_johnson',
   YeoJohnsonTransformer(variables=['cont0', 'cont1', 'cont2', 'cont6', 'cont8',
               

In [6]:
X_ = base_line.fit_transform(X, Y)
train_x, test_x, train_y, test_y = train_test_split(X_, Y, test_size=0.3)

C:\Users\hewar\AppData\Local\Programs\Python\Python39\lib\site-packages\feature_engine\encoding\rare_label.py:179: UserWarning: The number of unique categories for variable cat0 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
C:\Users\hewar\AppData\Local\Programs\Python\Python39\lib\site-packages\feature_engine\encoding\rare_label.py:179: UserWarning: The number of unique categories for variable cat1 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
C:\Users\hewar\AppData\Local\Programs\Python\Python39\lib\site-packages\feature_engine\encoding\rare_label.py:179: UserWarning: The number of unique categories for variable cat2 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


In [7]:
cv = RepeatedKFold(n_splits=8, n_repeats=16)

- linear models

In [9]:
elastic_net = Pipeline(steps=[('poly', PolynomialFeatures()),
                              ('elasticNet', ElasticNet())])
elastic_net.get_params()

{'memory': None,
 'steps': [('poly', PolynomialFeatures()), ('elasticNet', ElasticNet())],
 'verbose': False,
 'poly': PolynomialFeatures(),
 'elasticNet': ElasticNet(),
 'poly__degree': 2,
 'poly__include_bias': True,
 'poly__interaction_only': False,
 'poly__order': 'C',
 'elasticNet__alpha': 1.0,
 'elasticNet__copy_X': True,
 'elasticNet__fit_intercept': True,
 'elasticNet__l1_ratio': 0.5,
 'elasticNet__max_iter': 1000,
 'elasticNet__normalize': 'deprecated',
 'elasticNet__positive': False,
 'elasticNet__precompute': False,
 'elasticNet__random_state': None,
 'elasticNet__selection': 'cyclic',
 'elasticNet__tol': 0.0001,
 'elasticNet__warm_start': False}

In [10]:
max_iter = 16

rgs = RandomizedSearchCV(estimator=elastic_net,
                         param_distributions={'poly__degree': [2, 3],
                                              'elasticNet__alpha': [0.0001, 0.01, 0.1, 0.5, 1, 10],
                                              'elasticNet__l1_ratio': [0, 0.25, 0.5, 0.75, 1],
                                              'elasticNet__max_iter': [10000, 10000, 1000]
                                              },
                         n_iter=max_iter,
                         cv=cv,
                         return_train_score=True,
                         scoring=[('r2', make_scorer(r2_score)),
                                  ('mse', make_scorer(mean_squared_error, greater_is_better=False))],
                         refit='r2',
                         n_jobs=-1)
rgs.get_params()

{'cv': RepeatedKFold(n_repeats=16, n_splits=8, random_state=None),
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('poly', PolynomialFeatures()),
  ('elasticNet', ElasticNet())],
 'estimator__verbose': False,
 'estimator__poly': PolynomialFeatures(),
 'estimator__elasticNet': ElasticNet(),
 'estimator__poly__degree': 2,
 'estimator__poly__include_bias': True,
 'estimator__poly__interaction_only': False,
 'estimator__poly__order': 'C',
 'estimator__elasticNet__alpha': 1.0,
 'estimator__elasticNet__copy_X': True,
 'estimator__elasticNet__fit_intercept': True,
 'estimator__elasticNet__l1_ratio': 0.5,
 'estimator__elasticNet__max_iter': 1000,
 'estimator__elasticNet__normalize': 'deprecated',
 'estimator__elasticNet__positive': False,
 'estimator__elasticNet__precompute': False,
 'estimator__elasticNet__random_state': None,
 'estimator__elasticNet__selection': 'cyclic',
 'estimator__elasticNet__tol': 0.0001,
 'estimator__elasticNet__warm_start': False,
 'estimator':

In [ ]:
rgs.fit(train_x, train_y)
